# DQL

In [8]:
import requests
from pprint import pprint
import json 
import codecs


endpoint_diffbot = "http://kg.diffbot.com/kg/dql_endpoint"


def get_token():
    with open("../.diffbot-token", "r") as f:
        return f.read().strip()


def query_and_save(query, output_fpath):
    data = {
        "token": get_token(),
        "query": query,
        "type": "query"}
    r = requests.get(endpoint_diffbot, params=data)

    with codecs.open(output_fpath, "w", "utf-8") as out:   
        out.write(json.dumps(json.loads(r.content)))
    print(output_fpath)
    
    
entity_types = ["AdministrativeArea", "Article", "Corporation",
                    "DegreeEntity", "EducationMajorEntity", "EducationalInstitution",
                   "EmploymentCategory", "Image", "Intangible", "Landmark", "LocalBusiness",
                   "Miscellaneous", "Organization", "Person", "Place", "Product", "Role",
                   "Skill", "Video"]

for entity_type in entity_types:
    query_and_save(
        query='type:{}'.format(entity_type),
        output_fpath="data/{}.json".format(entity_type))
    
query_and_save(
    query='type:Person name:"Alexander Panchenko"',
    output_fpath="data/ap.json")


query_and_save(
    query='type:Person employments.employer.name:"Diffbot"',
    output_fpath="data/diffbot-employees.json")


query_and_save(
    query='type:Person employments.{title:"CEO" employer.name:"Diffbot"}',
    output_fpath="data/diffbot-ceo.json")

query_and_save(
    query='type:Person employments.{employer.name:"Diffbot" isCurrent:true}',
    output_fpath="data/diffbot-current-employees.json")



data/AdministrativeArea.json
data/Article.json
data/Corporation.json
data/DegreeEntity.json
data/EducationMajorEntity.json
data/EducationInstitution.json
data/EmploymentCategory.json
data/Image.json
data/Intangible.json
data/Landmark.json
data/LocalBusiness.json
data/Miscellaneous.json
data/Organization.json
data/Person.json
data/Place.json
data/Product.json
data/Role.json
data/Skill.json
data/Video.json
data/ap.json
data/diffbot-employees.json
data/diffbot-ceo.json
data/diffbot-current-employees.json


# Enhance API